# Sequential DQN

In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

>**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

# Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
import numpy as np
state = env.reset()
for _ in range(10):
    # env.render()
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action) # take a random action
    print('state, action, next_state, reward, done, info:', state, action, next_state, reward, done, info)
    state = next_state
    if done:
        state = env.reset()

state, action, next_state, reward, done, info: [-0.04296398 -0.03105276 -0.02344146  0.00339839] 0 [-0.04358504 -0.22583082 -0.0233735   0.28859406] 1.0 False {}
state, action, next_state, reward, done, info: [-0.04358504 -0.22583082 -0.0233735   0.28859406] 1 [-0.04810166 -0.03038349 -0.01760161 -0.01136804] 1.0 False {}
state, action, next_state, reward, done, info: [-0.04810166 -0.03038349 -0.01760161 -0.01136804] 1 [-0.04870933  0.16498641 -0.01782898 -0.3095521 ] 1.0 False {}
state, action, next_state, reward, done, info: [-0.04870933  0.16498641 -0.01782898 -0.3095521 ] 0 [-0.0454096  -0.02987704 -0.02402002 -0.02254483] 1.0 False {}
state, action, next_state, reward, done, info: [-0.0454096  -0.02987704 -0.02402002 -0.02254483] 1 [-0.04600714  0.165581   -0.02447091 -0.32270852] 1.0 False {}
state, action, next_state, reward, done, info: [-0.04600714  0.165581   -0.02447091 -0.32270852] 1 [-0.04269552  0.36104271 -0.03092508 -0.62300704] 1.0 False {}
state, action, next_state, r

To shut the window showing the simulation, use `env.close()`.

If you ran the simulation above, we can look at the rewards:

In [4]:
# print(rewards[-20:])
# print(np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
# print(np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
# print(np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
# print(np.max(np.array(rewards)), np.min(np.array(rewards)))
# print(np.max(np.array(states)), np.min(np.array(states)))

The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [5]:
def model_input(state_size, lstm_size, batch_size=1):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    # RNN
    gru = tf.nn.rnn_cell.GRUCell(lstm_size)
    cell = tf.nn.rnn_cell.MultiRNNCell([gru], state_is_tuple=False)
    initial_state = cell.zero_state(batch_size, tf.float32)
    return states, actions, targetQs, cell, initial_state

In [6]:
# RNN generator or sequence generator
def generator(states, num_classes, initial_state, cell, lstm_size, reuse=False): 
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        inputs = tf.layers.dense(inputs=states, units=lstm_size)
        print(states.shape, inputs.shape)
        
        # with tf.variable_scope('dynamic_rnn_', reuse=tf.AUTO_REUSE):
        # dynamic means adapt to the batch_size
        inputs_rnn = tf.reshape(inputs, [1, -1, lstm_size]) # NxH -> 1xNxH
        print(inputs_rnn.shape, initial_state.shape)
        outputs_rnn, final_state = tf.nn.dynamic_rnn(cell=cell, inputs=inputs_rnn, initial_state=initial_state)
        print(outputs_rnn.shape, final_state.shape)
        outputs = tf.reshape(outputs_rnn, [-1, lstm_size]) # 1xNxH -> NxH
        print(outputs.shape)

        # Last fully connected layer
        logits = tf.layers.dense(inputs=outputs, units=num_classes)
        print(logits.shape)
        #predictions = tf.nn.softmax(logits)
        
        # logits are the action logits
        return logits, final_state

In [7]:
def model_loss(action_size, hidden_size, states, cell, initial_state, actions, targetQs):
    actions_logits, final_state = generator(states=states, cell=cell, initial_state=initial_state, 
                                            lstm_size=hidden_size, num_classes=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    Qs = tf.reduce_max(actions_logits*actions_labels, axis=1)
    loss = tf.reduce_mean(tf.square(Qs - targetQs))
    return actions_logits, final_state, loss

In [8]:
def model_opt(loss, learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]

    # # Optimize
    # with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
    # #opt = tf.train.AdamOptimizer(learning_rate).minimize(loss, var_list=g_vars)

    #grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(loss, g_vars), clip_norm=5) # usually around 1-5
    grads = tf.gradients(loss, g_vars)
    opt = tf.train.AdamOptimizer(learning_rate).apply_gradients(grads_and_vars=zip(grads, g_vars))

    return opt

In [9]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, cell, self.initial_state = model_input(
            state_size=state_size, lstm_size=hidden_size)
        
        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.final_state, self.loss = model_loss(
            action_size=action_size, hidden_size=hidden_size, 
            states=self.states, actions=self.actions, 
            targetQs=self.targetQs, cell=cell, initial_state=self.initial_state)

        # Update the model: backward pass and backprop
        self.opt = model_opt(loss=self.loss, learning_rate=learning_rate)

## Experience replay

Reinforcement learning algorithms can have stability issues due to correlations between states. To reduce correlations when training, we can store the agent's experiences and later draw a random mini-batch of those experiences to train on. 

Here, we'll create a `Memory` object that will store our experiences, our transitions $<s, a, r, s'>$. This memory will have a maxmium capacity, so we can keep newer experiences in memory while getting rid of older experiences. Then, we'll sample a random mini-batch of transitions $<s, a, r, s'>$ and train on those.

Below, I've implemented a `Memory` object. If you're unfamiliar with `deque`, this is a double-ended queue. You can think of it like a tube open on both sides. You can put objects in either side of the tube. But if it's full, adding anything more will push an object out the other side. This is a great data structure to use for the memory buffer.

In [10]:
from collections import deque

class Memory():    
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size)
        self.states = deque(maxlen=max_size)
    def sample(self, batch_size):
        idx = np.random.choice(np.arange(len(self.buffer)), 
                               size=batch_size, 
                               replace=False)
        return [self.buffer[ii] for ii in idx], [self.states[ii] for ii in idx]

## Exploration - Exploitation

To learn about the environment and rules of the game, the agent needs to explore by taking random actions. We'll do this by choosing a random action with some probability $\epsilon$ (epsilon).  That is, with some probability $\epsilon$ the agent will make a random action and with probability $1 - \epsilon$, the agent will choose an action from $Q(s,a)$. This is called an **$\epsilon$-greedy policy**.


At first, the agent needs to do a lot of exploring. Later when it has learned more, the agent can favor choosing actions based on what it has learned. This is called _exploitation_. We'll set it up so the agent is more likely to explore early in training, then more likely to exploit later in training.

## Q-Learning training algorithm

Putting all this together, we can list out the algorithm we'll use to train the network. We'll train the network in _episodes_. One *episode* is one simulation of the game. For this game, the goal is to keep the pole upright for 195 frames. So we can start a new episode once meeting that goal. The game ends if the pole tilts over too far, or if the cart moves too far the left or right. When a game ends, we'll start a new episode. Now, to train the agent:

* Initialize the memory $D$
* Initialize the action-value network $Q$ with random weights
* **For** episode = 1, $M$ **do**
  * **For** $t$, $T$ **do**
     * With probability $\epsilon$ select a random action $a_t$, otherwise select $a_t = \mathrm{argmax}_a Q(s,a)$
     * Execute action $a_t$ in simulator and observe reward $r_{t+1}$ and new state $s_{t+1}$
     * Store transition $<s_t, a_t, r_{t+1}, s_{t+1}>$ in memory $D$
     * Sample random mini-batch from $D$: $<s_j, a_j, r_j, s'_j>$
     * Set $\hat{Q}_j = r_j$ if the episode ends at $j+1$, otherwise set $\hat{Q}_j = r_j + \gamma \max_{a'}{Q(s'_j, a')}$
     * Make a gradient descent step with loss $(\hat{Q}_j - Q(s_j, a_j))^2$
  * **endfor**
* **endfor**

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [11]:
# print('state:', np.array(states).shape[1], 
#       'action size: {}'.format((np.max(np.array(actions)) - np.min(np.array(actions)))+1))

In [12]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
action_size = 2
state_size = 4
hidden_size = 64               # number of units in each Q-network hidden layer
learning_rate = 0.0001         # Q-network learning rate

# Memory parameters
memory_size = 256            # memory capacity - 1000 DQN
batch_size = 256             # experience mini-batch size - 20 DQN
gamma = 0.99                 # future reward discount

In [13]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

# Init the memory
memory = Memory(max_size=batch_size)

(?, 4) (?, 64)
(1, ?, 64) (1, 64)
(1, ?, 64) (1, 64)
(?, 64)
(?, 2)


## Populate the memory (exprience memory)

Here I'm re-initializing the simulation and pre-populating the memory. The agent is taking random actions and storing the transitions in memory. This will help the agent with exploring the game.

In [14]:
state = env.reset()
for _ in range(batch_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    memory.buffer.append([state, action, next_state, reward, float(done)])
    memory.states.append(np.zeros([1, hidden_size]))
    state = next_state
    if done is True:
        # Reseting the env/first state
        state = env.reset()

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
saver = tf.train.Saver()
episode_rewards_list, rewards_list, loss_list = [], [], []

# TF session for training
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    episode_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
    
    # Training episodes/epochs
    for ep in range(11111):
        total_reward = 0
        loss_batch = []
        state = env.reset()
        initial_state = sess.run(model.initial_state)

        # Training steps/batches
        while True:
            action_logits, final_state = sess.run([model.actions_logits, model.final_state],
                                                  feed_dict = {model.states: state.reshape([1, -1]), 
                                                               model.initial_state: initial_state})
            action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done)])
            memory.states.append(initial_state)
            total_reward += reward
            state = next_state
            initial_state = final_state

            # Training
            batch = memory.buffer
            states = np.array([each[0] for each in batch])
            actions = np.array([each[1] for each in batch])
            next_states = np.array([each[2] for each in batch])
            rewards = np.array([each[3] for each in batch])
            dones = np.array([each[4] for each in batch])
            initial_states = memory.states
            #print(len(initial_states))
            next_actions_logits = sess.run(model.actions_logits, 
                                           feed_dict = {model.states: next_states,
                                                        model.initial_state: initial_states[1]})
            nextQs = np.max(next_actions_logits, axis=1) * (1-dones)
            targetQs = rewards + (gamma * nextQs)
            loss, _ = sess.run([model.loss, model.opt], feed_dict = {model.states: states, 
                                                                     model.actions: actions,
                                                                     model.targetQs: targetQs,
                                                                     model.initial_state: initial_states[0]})
            loss_batch.append(loss)
            if done is True:
                break
                
        # Outputing: priting out/Potting
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'loss:{:.4f}'.format(np.mean(loss_batch)))
        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        loss_list.append([ep, np.mean(loss_batch)])
        # Break episode/epoch loop
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:72.0000 R:72.0000 loss:1.4937
Episode:1 meanR:90.0000 R:108.0000 loss:2.4562
Episode:2 meanR:85.6667 R:77.0000 loss:1.3230
Episode:3 meanR:83.0000 R:75.0000 loss:1.3072
Episode:4 meanR:89.2000 R:114.0000 loss:1.4993
Episode:5 meanR:82.6667 R:50.0000 loss:1.6534
Episode:6 meanR:77.2857 R:45.0000 loss:1.8867
Episode:7 meanR:70.6250 R:24.0000 loss:2.1576
Episode:8 meanR:65.8889 R:28.0000 loss:2.5613
Episode:9 meanR:61.9000 R:26.0000 loss:2.7019
Episode:10 meanR:58.5455 R:25.0000 loss:3.1468
Episode:11 meanR:55.8333 R:26.0000 loss:3.6171
Episode:12 meanR:53.5385 R:26.0000 loss:4.1480
Episode:13 meanR:51.1429 R:20.0000 loss:4.5446
Episode:14 meanR:48.8000 R:16.0000 loss:4.8331
Episode:15 meanR:46.7500 R:16.0000 loss:5.3477
Episode:16 meanR:45.1765 R:20.0000 loss:5.5865
Episode:17 meanR:43.8333 R:21.0000 loss:5.9231
Episode:18 meanR:42.4211 R:17.0000 loss:6.3264
Episode:19 meanR:41.2000 R:18.0000 loss:6.5855
Episode:20 meanR:39.9048 R:14.0000 loss:6.7393
Episode:21 meanR:38.7

Episode:172 meanR:18.8400 R:73.0000 loss:4.6331
Episode:173 meanR:19.3300 R:67.0000 loss:4.9457
Episode:174 meanR:19.6800 R:50.0000 loss:3.0377
Episode:175 meanR:19.9700 R:43.0000 loss:2.8979
Episode:176 meanR:20.4600 R:65.0000 loss:2.5510
Episode:177 meanR:20.5300 R:25.0000 loss:4.1885
Episode:178 meanR:20.6100 R:23.0000 loss:5.8819
Episode:179 meanR:20.6400 R:22.0000 loss:6.4060
Episode:180 meanR:20.8700 R:39.0000 loss:6.1961
Episode:181 meanR:20.9900 R:28.0000 loss:5.7168
Episode:182 meanR:21.1400 R:29.0000 loss:5.8939
Episode:183 meanR:21.3000 R:33.0000 loss:6.0657
Episode:184 meanR:21.3900 R:23.0000 loss:5.9176
Episode:185 meanR:21.5500 R:33.0000 loss:6.3248
Episode:186 meanR:21.7200 R:34.0000 loss:6.2037
Episode:187 meanR:21.8600 R:32.0000 loss:6.0078
Episode:188 meanR:22.1800 R:51.0000 loss:6.1007
Episode:189 meanR:22.3400 R:31.0000 loss:5.8923
Episode:190 meanR:22.4300 R:25.0000 loss:4.9543
Episode:191 meanR:23.1000 R:83.0000 loss:3.3565
Episode:192 meanR:23.2500 R:31.0000 loss

Episode:340 meanR:93.6700 R:115.0000 loss:10.2869
Episode:341 meanR:94.2600 R:113.0000 loss:6.7593
Episode:342 meanR:94.8500 R:105.0000 loss:3.5391
Episode:343 meanR:95.6300 R:124.0000 loss:2.8368
Episode:344 meanR:96.4400 R:114.0000 loss:1.8821
Episode:345 meanR:97.4200 R:142.0000 loss:1.9801
Episode:346 meanR:98.4200 R:133.0000 loss:1.4097
Episode:347 meanR:99.1700 R:120.0000 loss:0.6926
Episode:348 meanR:100.4000 R:162.0000 loss:1.9768
Episode:349 meanR:101.4500 R:144.0000 loss:4.6269
Episode:350 meanR:102.3500 R:126.0000 loss:3.8050
Episode:351 meanR:103.3800 R:148.0000 loss:1.1870
Episode:352 meanR:104.5300 R:169.0000 loss:0.8451
Episode:353 meanR:106.0600 R:212.0000 loss:14.3377
Episode:354 meanR:105.8500 R:43.0000 loss:11.5505
Episode:355 meanR:105.4500 R:34.0000 loss:16.9017
Episode:356 meanR:105.3400 R:56.0000 loss:28.1484
Episode:357 meanR:105.9800 R:144.0000 loss:13.5926
Episode:358 meanR:106.1600 R:129.0000 loss:4.9282
Episode:359 meanR:107.2400 R:255.0000 loss:3.9645
Episo

Episode:504 meanR:76.8600 R:500.0000 loss:61.6985
Episode:505 meanR:80.4200 R:500.0000 loss:14.2678
Episode:506 meanR:83.5600 R:500.0000 loss:18.7064
Episode:507 meanR:87.4000 R:500.0000 loss:17.2428
Episode:508 meanR:91.3200 R:500.0000 loss:17.2600
Episode:509 meanR:95.1400 R:500.0000 loss:16.1911
Episode:510 meanR:98.4500 R:500.0000 loss:4.9404
Episode:511 meanR:100.5900 R:500.0000 loss:12.7276
Episode:512 meanR:104.9800 R:500.0000 loss:6.1105
Episode:513 meanR:109.5200 R:500.0000 loss:17.2936
Episode:514 meanR:113.8600 R:500.0000 loss:12.9789
Episode:515 meanR:118.2700 R:500.0000 loss:17.3895
Episode:516 meanR:118.2700 R:500.0000 loss:17.0008
Episode:517 meanR:122.6400 R:500.0000 loss:15.4533
Episode:518 meanR:126.6900 R:500.0000 loss:6.9199
Episode:519 meanR:129.5600 R:500.0000 loss:19.5019
Episode:520 meanR:134.0400 R:500.0000 loss:17.9001
Episode:521 meanR:138.3400 R:500.0000 loss:18.2933
Episode:522 meanR:142.2600 R:500.0000 loss:4.0767
Episode:523 meanR:145.0400 R:328.0000 loss

Episode:666 meanR:475.3500 R:157.0000 loss:37.7182
Episode:667 meanR:473.7200 R:337.0000 loss:28.7577
Episode:668 meanR:470.5500 R:183.0000 loss:23.7105
Episode:669 meanR:470.5500 R:500.0000 loss:6.0080
Episode:670 meanR:470.5500 R:500.0000 loss:19.0469
Episode:671 meanR:468.1500 R:260.0000 loss:39.7077
Episode:672 meanR:465.3400 R:219.0000 loss:2.0719
Episode:673 meanR:462.5600 R:222.0000 loss:2.0398
Episode:674 meanR:459.4900 R:193.0000 loss:3.1662
Episode:675 meanR:456.3500 R:186.0000 loss:1.6247
Episode:676 meanR:453.3800 R:203.0000 loss:2.2476
Episode:677 meanR:450.3100 R:193.0000 loss:2.9433
Episode:678 meanR:446.4900 R:118.0000 loss:1.8950
Episode:679 meanR:442.7600 R:127.0000 loss:20.7684
Episode:680 meanR:439.0300 R:127.0000 loss:27.1335
Episode:681 meanR:436.3700 R:234.0000 loss:17.5458
Episode:682 meanR:433.8300 R:246.0000 loss:4.7755
Episode:683 meanR:433.8300 R:500.0000 loss:1.2746
Episode:684 meanR:433.8300 R:500.0000 loss:18.0467
Episode:685 meanR:433.8300 R:500.0000 los

...

Episode:1468 meanR:301.5200 R:61.0000 loss:17.1794
Episode:1469 meanR:301.0000 R:67.0000 loss:15.2987
Episode:1470 meanR:296.6100 R:61.0000 loss:11.9238
Episode:1471 meanR:292.4300 R:82.0000 loss:7.4370
Episode:1472 meanR:287.9600 R:53.0000 loss:5.5110
Episode:1473 meanR:283.8100 R:85.0000 loss:4.7695
Episode:1474 meanR:279.3300 R:52.0000 loss:4.7644
Episode:1475 meanR:274.7700 R:44.0000 loss:4.2998
Episode:1476 meanR:270.2000 R:43.0000 loss:4.4627
Episode:1477 meanR:265.5600 R:36.0000 loss:4.9829
Episode:1478 meanR:260.8700 R:31.0000 loss:5.1833
Episode:1479 meanR:256.1300 R:26.0000 loss:5.8243
Episode:1480 meanR:251.4300 R:30.0000 loss:6.8016
Episode:1481 meanR:246.8100 R:38.0000 loss:7.3144
Episode:1482 meanR:242.1900 R:38.0000 loss:9.4443
Episode:1483 meanR:237.4900 R:30.0000 loss:9.4724
Episode:1484 meanR:232.7800 R:29.0000 loss:9.9015
Episode:1485 meanR:228.0300 R:25.0000 loss:8.8569
Episode:1486 meanR:223.3000 R:27.0000 loss:9.8632
Episode:1487 meanR:218.5700 R:27.0000 loss:8.40

Episode:1627 meanR:406.0500 R:500.0000 loss:19.9865
Episode:1628 meanR:406.0500 R:500.0000 loss:17.2423
Episode:1629 meanR:406.0500 R:500.0000 loss:62.7218
Episode:1630 meanR:406.0500 R:500.0000 loss:20.6413
Episode:1631 meanR:406.0500 R:500.0000 loss:21.7921
Episode:1632 meanR:406.0500 R:500.0000 loss:20.8083
Episode:1633 meanR:406.0500 R:500.0000 loss:20.7595
Episode:1634 meanR:406.0500 R:500.0000 loss:33.4078
Episode:1635 meanR:406.0500 R:500.0000 loss:21.2854
Episode:1636 meanR:406.0500 R:500.0000 loss:21.8564
Episode:1637 meanR:406.0500 R:500.0000 loss:21.2249
Episode:1638 meanR:406.0500 R:500.0000 loss:20.3106
Episode:1639 meanR:406.0500 R:500.0000 loss:19.3968
Episode:1640 meanR:406.0500 R:500.0000 loss:18.4955
Episode:1641 meanR:406.0500 R:500.0000 loss:19.7489
Episode:1642 meanR:406.0500 R:500.0000 loss:19.0671
Episode:1643 meanR:406.0500 R:500.0000 loss:18.7052
Episode:1644 meanR:406.0500 R:500.0000 loss:21.3171
Episode:1645 meanR:406.0500 R:500.0000 loss:23.1056
Episode:1646

Episode:1786 meanR:366.2000 R:284.0000 loss:7.8293
Episode:1787 meanR:366.2000 R:500.0000 loss:2.8446
Episode:1788 meanR:366.2000 R:500.0000 loss:17.9561
Episode:1789 meanR:366.2000 R:500.0000 loss:14.2801
Episode:1790 meanR:366.2000 R:500.0000 loss:14.4467
Episode:1791 meanR:366.2000 R:500.0000 loss:15.9347
Episode:1792 meanR:366.2000 R:500.0000 loss:16.9578
Episode:1793 meanR:366.2000 R:500.0000 loss:14.7210
Episode:1794 meanR:366.2000 R:500.0000 loss:18.5230
Episode:1795 meanR:366.3400 R:278.0000 loss:163.1893
Episode:1796 meanR:367.4000 R:117.0000 loss:183.7857
Episode:1797 meanR:367.7100 R:108.0000 loss:75.6951
Episode:1798 meanR:363.7400 R:103.0000 loss:43.5769
Episode:1799 meanR:359.8100 R:107.0000 loss:45.5576
Episode:1800 meanR:356.0200 R:121.0000 loss:25.6483
Episode:1801 meanR:352.1500 R:113.0000 loss:33.6950
Episode:1802 meanR:348.0900 R:94.0000 loss:27.3934
Episode:1803 meanR:343.7500 R:66.0000 loss:34.8485
Episode:1804 meanR:339.4700 R:72.0000 loss:54.3290
Episode:1805 me

Episode:1944 meanR:457.4800 R:500.0000 loss:10.7419
Episode:1945 meanR:454.1900 R:171.0000 loss:23.7518
Episode:1946 meanR:454.1900 R:500.0000 loss:8.8971
Episode:1947 meanR:454.1900 R:500.0000 loss:18.0943
Episode:1948 meanR:454.1900 R:500.0000 loss:19.0866
Episode:1949 meanR:454.1900 R:500.0000 loss:15.8417
Episode:1950 meanR:454.1900 R:500.0000 loss:16.3641
Episode:1951 meanR:454.1900 R:500.0000 loss:23.9875
Episode:1952 meanR:454.1900 R:500.0000 loss:22.4341
Episode:1953 meanR:454.1900 R:500.0000 loss:21.7234
Episode:1954 meanR:454.1900 R:500.0000 loss:20.5517
Episode:1955 meanR:454.1900 R:500.0000 loss:19.2463
Episode:1956 meanR:456.2600 R:500.0000 loss:18.2266
Episode:1957 meanR:461.1400 R:500.0000 loss:17.7744
Episode:1958 meanR:461.1400 R:500.0000 loss:26.6792
Episode:1959 meanR:461.1400 R:500.0000 loss:69.7036
Episode:1960 meanR:461.1400 R:500.0000 loss:15.8545
Episode:1961 meanR:461.1400 R:500.0000 loss:17.8678
Episode:1962 meanR:461.1400 R:500.0000 loss:21.9181
Episode:1963 

Episode:2103 meanR:358.0500 R:108.0000 loss:1.2768
Episode:2104 meanR:357.1500 R:123.0000 loss:1.6739
Episode:2105 meanR:355.7900 R:100.0000 loss:2.2329
Episode:2106 meanR:354.9600 R:104.0000 loss:2.2893
Episode:2107 meanR:354.3800 R:118.0000 loss:2.2480
Episode:2108 meanR:353.3300 R:92.0000 loss:2.3241
Episode:2109 meanR:349.2700 R:94.0000 loss:3.3105
Episode:2110 meanR:345.1500 R:88.0000 loss:3.4914
Episode:2111 meanR:341.0100 R:86.0000 loss:2.9139
Episode:2112 meanR:336.7800 R:77.0000 loss:2.4886
Episode:2113 meanR:332.5800 R:80.0000 loss:2.4478
Episode:2114 meanR:328.3900 R:81.0000 loss:2.2519
Episode:2115 meanR:324.2400 R:85.0000 loss:3.0214
Episode:2116 meanR:320.1100 R:87.0000 loss:2.7362
Episode:2117 meanR:316.0000 R:89.0000 loss:1.5043
Episode:2118 meanR:311.8200 R:82.0000 loss:1.9470
Episode:2119 meanR:307.7000 R:88.0000 loss:2.1536
Episode:2120 meanR:303.6100 R:91.0000 loss:1.7224
Episode:2121 meanR:299.4400 R:83.0000 loss:1.5048
Episode:2122 meanR:295.2800 R:84.0000 loss:1.

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Average losses')

## Testing

Let's checkout how our trained agent plays the game.

In [184]:
import gym

# Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model-seq.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    initial_state = sess.run(model.initial_state) # Qs or current batch or states[:-1]
    
    # Episode/epoch
    for _ in range(1):
        state = env.reset()
        total_reward = 0
        
        # Steps/batches
        while True:
            env.render()
            action_logits, initial_state = sess.run([model.actions_logits, model.final_state],
                                                    feed_dict = {model.states: state.reshape([1, -1]), 
                                                                 model.initial_state: initial_state})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # At the end of each episode
        print('total_reward:{}'.format(total_reward))

# Close the env
env.close()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


total_reward:120.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.